In [6]:
import numpy as np
import gradio as gr
from PIL import Image
from numpy import asarray
from tensorflow.keras.models import load_model
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [7]:
class_names = ['Bouvier Des Flandres', 'Standard Poodle', 'Welsh Springer Spaniel', 'Norwegian Elkhound', 'Appenzeller', 'Airedale', 'Border Collie', 'Wire-Haired Fox Terrier', 'Affenpinscher', 'Siberian Husky', 'Australian Terrier', 'Chow', 'Leonberg', 'English Setter', 'Boxer', 'Toy Terrier', 'Rhodesian Ridgeback', 'Bluetick', 'Norwich Terrier', 'Basset', 'Bloodhound', 'Great Dane', 'Afghan Hound', 'Dhole', 'Basenji', 'German Shepherd', 'Sealyham Terrier', 'Sussex Spaniel', 'Old English Sheepdog', 'Keeshond', 'Shetland Sheepdog', 'Komondor', 'Shih-Tzu', 'Entlebucher', 'Irish Wolfhound', 'Lhasa', 'Black-And-Tan Coonhound', 'Ibizan Hound', 'Irish Water Spaniel', 'German Short-Haired Pointer', 'Blenheim Spaniel', 'Scotch Terrier', 'Rottweiler', 'Mexican Hairless', 'Toy Poodle', 'Pug', 'Malinois', 'Golden Retriever', 'Curly-Coated Retriever', 'Maltese Dog', 'Miniature Poodle', 'Eskimo Dog', 'Briard', 'Bernese Mountain Dog', 'Border Terrier', 'Scottish Deerhound', 'French Bulldog', 'English Springer', 'Papillon', 'Labrador Retriever', 'Groenendael', 'American Staffordshire Terrier', 'Saluki', 'Kerry Blue Terrier', 'Miniature Pinscher', 'West Highland White Terrier', 'Staffordshire Bullterrier', 'English Foxhound', 'Dingo', 'Lakeland Terrier', 'Great Pyrenees', 'Standard Schnauzer', 'Walker Hound', 'Collie', 'Kuvasz', 'Flat-Coated Retriever', 'Bull Mastiff', 'African Hunting Dog', 'Chihuahua', 'Schipperke', 'Italian Greyhound', 'Silky Terrier', 'Cairn', 'Brabancon Griffon', 'Gordon Setter', 'Miniature Schnauzer', 'Weimaraner', 'Japanese Spaniel', 'Dandie Dinmont', 'Beagle', 'Brittany Spaniel', 'Newfoundland', 'Cardigan', 'Yorkshire Terrier', 'Vizsla', 'Redbone', 'Malamute', 'Clumber', 'Borzoi', 'Boston Bull', 'Tibetan Mastiff', 'Pomeranian', 'Tibetan Terrier', 'Doberman', 'Irish Setter', 'Kelpie', 'Norfolk Terrier', 'Greater Swiss Mountain Dog', 'Bedlington Terrier', 'Giant Schnauzer', 'Otterhound', 'Whippet', 'Irish Terrier', 'Soft-Coated Wheaten Terrier', 'Saint Bernard', 'Samoyed', 'Chesapeake Bay Retriever', 'Pembroke', 'Pekinese', 'Cocker Spaniel']
MODEL_PATH = '/content/drive/MyDrive/Machine Learning Projects/Dog Breed classification project/Deploy Model/model_ft_results(layers=75, lr=1e-06)v2.keras'
model = load_model(MODEL_PATH)

In [8]:
def predict(img):
  img = Image.fromarray(img)

  img = img.convert('RGB')
  img = img.resize((224, 224))
  img_array = np.asarray(img) / 255.0
  img_array = np.expand_dims(img_array, axis=0)

  y_hat = model.predict(img_array).flatten()
  top3 = np.argsort(y_hat)[-3:][::-1]

  return get_html(class_names, y_hat, top_k=3)


In [9]:
def get_html(class_names, y_hat, top_k=3):
    top_indices = np.argsort(y_hat)[-top_k:][::-1]
    medals = ['🥇', '🥈', '🥉']

    html = """
    <style>
        @keyframes gradientShift {
            0% { background-position: 0% 50%; }
            50% { background-position: 100% 50%; }
            100% { background-position: 0% 50%; }
        }

        .animated-card {
            padding: 15px;
            border-radius: 12px;
            box-shadow: 0 4px 15px rgba(0,0,0,0.15);
            background: linear-gradient(-45deg, #e65100, #ff8f00, #ff6f00, #e65100);
            background-size: 300% 300%;
            animation: gradientShift 5s ease infinite;
            color: white;
        }

        .wiki-link {
            font-size: 13px;
            color: rgba(255, 255, 255, 0.8);
            text-decoration: none;
        }

        .wiki-link:hover {
            text-decoration: underline;
        }
    </style>

    <div style="display: flex; flex-direction: column; gap: 15px; font-family: 'Segoe UI', sans-serif;">
    """

    for idx, i in enumerate(top_indices):
        breed = class_names[i]
        prob = y_hat[i]
        medal = medals[idx] if idx < len(medals) else ''

        # Format breed name for Wikipedia URL
        wiki_query = breed.replace(' ', '_')
        wiki_link = f"https://en.wikipedia.org/wiki/{wiki_query}"

        html += f"""
        <div class="animated-card">
            <div style="font-size: 22px; font-weight: 700; margin-bottom: 8px;">
                {medal} {breed}
            </div>
            <div style="height: 10px; background: rgba(255,255,255,0.3); border-radius: 5px; overflow: hidden;">
                <div style="width: {prob*100:.1f}%; height: 100%; background: white;"></div>
            </div>
            <div style="margin-top: 5px; font-size: 15px; display: flex; justify-content: space-between; align-items: center;">
                <span>Confidence: <b>{prob:.2%}</b></span>
                <a href="{wiki_link}" target="_blank" class="wiki-link">more into</a>
            </div>
        </div>
        """

    html += "</div>"
    return html

In [10]:
demo = gr.Interface(
    predict,
    inputs=gr.Image(width=244, height=244, image_mode='RGB'),
    outputs=gr.HTML(),
    title="🐾 Dog Breed Classifier",
    description="📸 Upload a photo of a dog to identify its breed!"
    )
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4dfa4eaf90a8172fbf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
